In [ ]:
library(data.table)
library(dplyr)
library(dtplyr)

In [ ]:
heads = "qseqid sseqid pident evalue bitscore qcovhsp staxid"
heads = unlist(strsplit(heads, split = " ")[[1]])
df_nr = fread(input = snakemake@input$vsbalt, header = F, showProgress = T)
names(df_nr) <- heads 
df_nr <- lazy_dt(df_nr)

In [ ]:
df_nr2 <- df_nr %>%
    filter(pident > 30, qcovhsp > 50) %>%
    group_by(qseqid) %>%
    arrange(-bitscore, .by_group = T) %>%
    slice(1) %>%
    ungroup() %>%
    select(qseqid,sseqid_nr = sseqid, pident_nr = pident,bitscore_nr = bitscore, staxid_nr = staxid, qcovhsp_nr = qcovhsp) #%>%
    #as.data.table()

In [ ]:
heads = "qseqid sseqid pident evalue bitscore qcovhsp"
heads = unlist(strsplit(heads, split = " ")[[1]])
df_bins = fread(input = snakemake@input$vsself, col.names = heads, showProgress = T)


In [ ]:
df_bins2 <- df_bins %>%
    rowwise() %>%
    mutate(
        qbin = strsplit(qseqid, split = "_")[[1]][[1]],
        sbin = strsplit(sseqid, split = "_")[[1]][[1]]
    ) %>%
    filter(qbin != sbin, pident > 50, qcovhsp > 50)
    
df_bins2 <- lazy_dt(df_bins2)
df_bins2 <- df_bins2 %>%
    left_join(df_nr2, by = "qseqid") %>%
    #filter(is.na(sseqid_nr)) %>%
    group_by(qseqid) %>%
    arrange(-bitscore, .by_group = T) %>%
    slice(1) %>%
    ungroup() %>%
    mutate(bitscore_nr = if_else(is.na(bitscore_nr),0,bitscore_nr)) %>%
    filter(bitscore >= bitscore_nr) %>%
    filter(!is.na(qseqid)) %>%
    collect()

fwrite(df_bins2,file = snakemake@output[[1]])